# Скачивание данных

In [40]:
# !pip install jsonlines


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [91]:
import jsonlines
import pandas as pd
import numpy as np

In [276]:
text = []
with jsonlines.open(r"Data\ranking_train.jsonl", 'r') as reader:
    for obj in  reader:
        text.append(obj)

In [210]:
'''Формат данных следующий... Сначала по ключу 'text' выдаёт тело поста; по ключу 'comments' получаем список, состоящий из словарей следующего вида - по ключу 'text' выдаёт 
тело комментария, а по ключу 'score' получаем оценку от 0 до 4'''
len(text) 

88107

In [211]:
print('Пример вызова тела поста: {}'.format(text[0]['text']) )

Пример вызова тела поста: How many summer Y Combinator fundees decided not to continue with their startup and go back to school? and what were the reasons?


In [192]:
text[0]

{'text': 'How many summer Y Combinator fundees decided not to continue with their startup and go back to school? and what were the reasons?',
 'comments': [{'text': 'Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there.  However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I\'d guess we\'ve had about 8 startups where the founders went back to school.  It doesn\'t only happen with summer batches.  Founders from winter batches do it too.Usually the reason is that the startup isn\'t doing very well. However, that judgement depends a lot on how determined the founders are.  One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups loo

In [121]:
print('Пример вызова тела комментариев: {}'.format(text[0]['comments']) )

Пример вызова тела комментариев: [{'text': 'Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there.  However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I\'d guess we\'ve had about 8 startups where the founders went back to school.  It doesn\'t only happen with summer batches.  Founders from winter batches do it too.Usually the reason is that the startup isn\'t doing very well. However, that judgement depends a lot on how determined the founders are.  One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups look bad after 3 months.  Someone who\'s out of school and has to make it work or get a job in a cubicle will say "don\'t wor

In [331]:
pd_data = pd.DataFrame(columns = ['post', 'comment', 'score'])
i=0
batch_size = 32
def bach_generator(batch_size, text):
    i=0
    batch_idx = np.random.random_integers(0, len(text), batch_size)
    
    data = np.array(text)[[batch_idx]][0]
    
    for body in data:
        for comment in body['comments']:
            pd_data.loc[i] = [body['text'].lower(), comment['text'].lower(), comment['score']]
            i+=1
    return pd_data

    

In [333]:
bach_generator(batch_size, text)

C:\Users\User\AppData\Local\Temp\ipykernel_10104\457461032.py:6: DeprecationWarning: This function is deprecated. Please call randint(0, 88107 + 1) instead
  batch_idx = np.random.random_integers(0, len(text), batch_size)


,post,comment,score
0,how to find hackers for only stock options?,"from first principles, actively looking for su...",0
1,how to find hackers for only stock options?,"i meet half of my current core team, buy not b...",1
2,how to find hackers for only stock options?,i had that same question 4-5 years ago when i ...,2
3,how to find hackers for only stock options?,someone who doesn't take a salary for six mont...,3
4,how to find hackers for only stock options?,i was a person you are looking for. two years ...,4
...,...,...,...
155,netherlands to tax drivers by the kilometer,the dutch government is batting 10 for 10 when...,0
156,netherlands to tax drivers by the kilometer,doesn't the gasoline tax we pay in the us acco...,1
157,netherlands to tax drivers by the kilometer,apart from hating the big brother aspects of t...,2
158,netherlands to tax drivers by the kilometer,it's a stupid approach and it will never see t...,3


In [251]:
'''Перегонка датафрейма в CSV. Этот файл уже висит в репо'''
# pd_data.to_csv (r'my_data.csv', index= False )

'Перегонка датафрейма в CSV. Этот файл уже висит в репо'

# Preprocessing

In [361]:
import string
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from functools import reduce

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [392]:
stop_w =stopwords.words("english")
ps = PorterStemmer()
pd_data_sent = pd.DataFrame(columns = ['post', 'comment', 'score'])
pd_data_int = pd.DataFrame(columns = ['post', 'comment', 'score'])
dictionary = []
i=0
batch_size = 32

def bach_generator(batch_size, text):
    i=0
    batch_idx = np.random.random_integers(0, len(text), batch_size)
    
    data = np.array(text)[[batch_idx]][0]
    
    for body in data:
        for comment in body['comments']:
            post_text = body['text'].translate(str.maketrans("", "", string.punctuation))
            post_tokens = word_tokenize(post_text)
            
            comment_text = comment['text'].translate(str.maketrans("", "", string.punctuation))
            comment_tokens = word_tokenize(comment_text)
            
            
            stemmed_post = [ps.stem(x) if x not in stop_w else '' for x in post_tokens ]
            stemmed_coment = [ps.stem(x) if x not in stop_w else '' for x in comment_tokens ]
            
            pd_data_sent.loc[i] = [' '.join(stemmed_post), ' '.join(stemmed_coment), comment['score']]
            pd_data_sent.loc[i] = [' '.join(stemmed_post), ' '.join(stemmed_coment), comment['score']]
            i+=1
            
    return pd_data

data = bach_generator(batch_size, text)


C:\Users\User\AppData\Local\Temp\ipykernel_10104\1020212156.py:8: DeprecationWarning: This function is deprecated. Please call randint(0, 88107 + 1) instead
  batch_idx = np.random.random_integers(0, len(text), batch_size)


In [393]:
data


,post,comment,score
0,how the hell is thi my fault,i cant believ im defend arrington i think ...,0
1,how the hell is thi my fault,everybodi calm pleaseej real she real perso...,1
2,how the hell is thi my fault,you know i cheerlead michael arrington weve...,2
3,how the hell is thi my fault,hn pretti disgust last day level schaden...,3
4,how the hell is thi my fault,sorri pg steep hill climb want defend ...,4
...,...,...,...
155,john carmack quakecon 2013 keynot livestream,quotkinect sort like zerobutton mous lot ...,0
156,john carmack quakecon 2013 keynot livestream,john carmack talk length haskel keynot def...,1
157,john carmack quakecon 2013 keynot livestream,he talk length recent year armadillo i thi...,2
158,john carmack quakecon 2013 keynot livestream,you could also watch carmackx27 keynot youtub...,3


In [394]:
train_target = data['score']
train_data = data['comment']

In [ ]:
from sklearn.naive_bayes import MultinomialNB